In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris,load_wine
from DecisionTree import D_Tree 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from stratifion import stratified_sampling
from MetaCost import Metacost
from ucimlrepo import fetch_ucirepo
from KNN_plus import KNN

In [2]:
def make_cost_matrix(label):
    # np.random.seed(0)
    label_list,counts = np.unique(label,return_counts=True)
    class_number = len(label_list)
    data = np.zeros((class_number,class_number))
    Data = pd.DataFrame(data,columns=label_list,index=label_list)
    for name1 in label_list:
        for name2 in label_list:
            if name1 == name2:
                Data[name1][name2] = np.random.randint(0,1000)
            else:
                Data[name1][name2] = np.random.randint(0,2000*counts[np.where(label_list==name1)] / counts[np.where(label_list==name2)])
                # Data[name1][name2] = np.random.uniform(0,10000)
    return Data

In [3]:
print(make_cost_matrix(['A','A','B','C','A','C','A']))

        A      B      C
A    35.0   77.0  901.0
B  5050.0  875.0  746.0
C  1898.0   86.0  834.0


In [4]:
def compute_cs(cost_matrix,true_label,predict_label):
    cs = 0

    for i in range(len(true_label)):
        
        # if true_label[i]!=predict_label[i]:
        cs += cost_matrix.iloc[predict_label[i]][true_label[i]]

    # conf_matrix = confusion_matrix(true_label, predict_label)
    # total_cost = np.sum(conf_matrix * cost_matrix.values)

    # return total_cost/len(true_label)
    return cs/len(true_label)
    

In [5]:
def compute_acc(true_label,predict_label):
    acc = accuracy_score(true_label,predict_label)
    return acc

In [6]:
# def compute_Auc(true_label,predict_label):
#     return roc_auc_score(true_label,predict_label)

In [7]:
def compute_f1(true_label,predict_label):
    return f1_score(true_label,predict_label,average='macro')

In [8]:
# iris数据集
iris = load_iris()
iris_data = iris.data
iris_label = iris.target
iris_ = pd.DataFrame(iris_data)
column = iris_.columns
iris_ = (iris_[column] - iris_[column].min()) / (iris_[column].max() - iris_[column].min())
iris_['Y'] = iris_label
cs = []


for i in range(20):
    cost_matrix = make_cost_matrix(iris_label)
    iris_train_set,iris_test_set = train_test_split(iris_,test_size=0.3,random_state=42)
    iris_test_data = iris_test_set.iloc[:,:-1].values
    iris_test_label = iris_test_set.iloc[:,-1].values
    tree = D_Tree(iris_train_set)
    tree.fit()
    predict_label = tree.predict(iris_test_data)
    cs.append(compute_cs(cost_matrix,iris_test_label,predict_label))
cs = np.array(cs)
# print(cs)
print("C4.5",np.mean(cs))
print("C4.5",np.std(cs))

cs_ = []
for i in range(20):
    cost_matrix = make_cost_matrix(iris_label)
    iris_train_set,iris_test_set = train_test_split(iris_,test_size=0.3,random_state=42)
    iris_test_data = iris_test_set.iloc[:,:-1].values
    iris_test_label = iris_test_set.iloc[:,-1].values
    train_set_x,train_set_y = stratified_sampling(iris_train_set,cost_matrix.values,'undersampling') 
    train_set = pd.DataFrame(train_set_x)
    train_set['Y'] = train_set_y
    tree = D_Tree(train_set)
    tree.fit()
    predict_label = tree.predict(iris_test_data)
    cs_.append(compute_cs(cost_matrix,iris_test_label,predict_label))
cs_ = np.array(cs_)
# print(cs_)
print("under",np.mean(cs_))
print("under",np.std(cs_))

cs__ = []
for i in range(20):
    cost_matrix = make_cost_matrix(iris_label)
    iris_train_set,iris_test_set = train_test_split(iris_,test_size=0.3,random_state=42)
    iris_test_data = iris_test_set.iloc[:,:-1].values
    iris_test_label = iris_test_set.iloc[:,-1].values
    train_set_x,train_set_y = stratified_sampling(iris_train_set,cost_matrix.values,'oversampling') 
    train_set = pd.DataFrame(train_set_x)
    train_set['Y'] = train_set_y
    tree = D_Tree(train_set)
    tree.fit()
    predict_label = tree.predict(iris_test_data)
    cs__.append(compute_cs(cost_matrix,iris_test_label,predict_label))
cs__ = np.array(cs__)
# print(cs__)
print("over",np.mean(cs__))
print("over",np.std(cs__))

cs___=[]
for i in range(20):
    cost_matrix = make_cost_matrix(iris_label)
    iris_train_set,iris_test_set = train_test_split(iris_,test_size=0.3,random_state=42)
    iris_test_data = iris_test_set.iloc[:,:-1].values
    iris_test_label = iris_test_set.iloc[:,-1].values
    tree = DecisionTreeClassifier(criterion='entropy')
    meta = Metacost(iris_train_set,tree,cost_matrix.values,q=False)
    model = meta.fit('Y')
    predict_label = model.predict(iris_test_data)
    cs___.append(compute_cs(cost_matrix,iris_test_label,predict_label))
cs___ = np.array(cs___)
# print(cs__)
print("meta",np.mean(cs___))
print("meta",np.std(cs___))

cs_k = []
for i in range(20):
    cost_matrix = make_cost_matrix(iris_label)
    iris_train_set,iris_test_set = train_test_split(iris_,test_size=0.3,random_state=42)
    iris_test_data = iris_test_set.iloc[:,:-1].values
    iris_test_label = iris_test_set.iloc[:,-1].values
    knn = KNN(2,iris_train_set,50)
    knn.fit(cost_matrix.values,n_neighbor=20,euclidean_features=[0,1,2,3])
    predict_label = knn.predict(iris_test_set)
    cs_k.append(compute_cs(cost_matrix,iris_test_label,predict_label))
cs_k = np.array(cs_k)
print("my_algorithm",np.mean(cs_k))
print("my_algorithm",np.std(cs_k))

C4.5 576.8511111111112
C4.5 201.76813818139172
under 476.83666666666676
under 179.5797134841843
over 497.2522222222222
over 141.69888753843352
meta 349.0966666666667
meta 151.15693705059826
my_algorithm 350.95666666666665
my_algorithm 151.51289355149177


In [9]:
# Wine数据集
wine = load_wine()
wine_data = wine.data
wine_label = wine.target
wine_ = pd.DataFrame(wine_data)
wine_['Y'] = wine_label
cs = []
for i in range(20):
    cost_matrix = make_cost_matrix(wine_label)

    wine_train_set,wine_test_set = train_test_split(wine_,test_size=0.3)
    wine_test_data = wine_test_set.iloc[:,:-1].values
    wine_test_label = wine_test_set.iloc[:,-1].values
    tree = D_Tree(wine_train_set)
    tree.fit()
    predict_label = tree.predict(wine_test_data)
    cs.append(compute_cs(cost_matrix,wine_test_label,predict_label))
cs = np.array(cs)
print("C4.5",cs.mean())
print("C4.5",cs.std())

cs_ = []
for i in range(20):
    cost_matrix = make_cost_matrix(wine_label)
    wine_train_set,wine_test_set = train_test_split(wine_,test_size=0.3,random_state=42)
    wine_test_data = wine_test_set.iloc[:,:-1].values
    wine_test_label = wine_test_set.iloc[:,-1].values
    train_set_x,train_set_y = stratified_sampling(wine_train_set,cost_matrix.values,'undersampling') 
    train_set = pd.DataFrame(train_set_x)
    train_set['Y'] = train_set_y
    tree = D_Tree(train_set)
    tree.fit()
    predict_label = tree.predict(wine_test_data)
    cs_.append(compute_cs(cost_matrix,wine_test_label,predict_label))
cs_ = np.array(cs_)
# print(cs_)
print("under",np.mean(cs_))
print("under",np.std(cs_))

cs__ = []
for i in range(20):
    cost_matrix = make_cost_matrix(wine_label)
    wine_train_set,wine_test_set = train_test_split(wine_,test_size=0.3,random_state=42)
    wine_test_data = wine_test_set.iloc[:,:-1].values
    wine_test_label = wine_test_set.iloc[:,-1].values
    train_set_x,train_set_y = stratified_sampling(wine_train_set,cost_matrix.values,'oversampling') 
    train_set = pd.DataFrame(train_set_x)
    train_set['Y'] = train_set_y
    tree = D_Tree(train_set)
    tree.fit()
    predict_label = tree.predict(wine_test_data)
    cs__.append(compute_cs(cost_matrix,wine_test_label,predict_label))
cs__ = np.array(cs__)
# print(cs__)
print("over",np.mean(cs__))
print("over",np.std(cs__))

cs___=[]
for i in range(20):
    cost_matrix = make_cost_matrix(wine_label)
    wine_train_set,wine_test_set = train_test_split(wine_,test_size=0.3,random_state=42)
    wine_test_data = wine_test_set.iloc[:,:-1].values
    wine_test_label = wine_test_set.iloc[:,-1].values
    tree = DecisionTreeClassifier(criterion='entropy')
    meta = Metacost(wine_train_set,tree,cost_matrix.values,q=False)
    model = meta.fit('Y')
    predict_label = model.predict(wine_test_data)
    cs___.append(compute_cs(cost_matrix,wine_test_label,predict_label))
cs___ = np.array(cs___)
# print(cs__)
print("meta",np.mean(cs___))
print("meta",np.std(cs___))

cs_k = []
for i in range(20):
    cost_matrix = make_cost_matrix(wine_label)
    wine_train_set,wine_test_set = train_test_split(wine_,test_size=0.3,random_state=42)
    wine_test_data = wine_test_set.iloc[:,:-1].values
    wine_test_label = wine_test_set.iloc[:,-1].values
    knn = KNN(3,wine_train_set,50)
    knn.fit(cost_matrix.values,n_neighbor=20,euclidean_features=[i for i in range(wine_train_set.shape[1]-1)])
    predict_label = knn.predict(wine_test_set)
    cs_k.append(compute_cs(cost_matrix,wine_test_label,predict_label))
cs_k = np.array(cs_k)
print("my_algorithm",np.mean(cs_k))
print("my_algorithm",np.std(cs_k))

C4.5 584.4481481481481
C4.5 112.32646188579307
under 518.2370370370371
under 127.281567322109
over 536.4212962962963
over 146.09400933513513
meta 412.0768518518519
meta 183.77790730612946
my_algorithm 426.2990740740741
my_algorithm 133.67565190942133


In [10]:
annealing = fetch_ucirepo(id=3) 
anneal_data = annealing.data.features.values
anneal_label = annealing.data.targets.values

unique = np.unique(anneal_label)

change = {unique[i]:i for i in range(len(unique))}

a = []
for i in anneal_label:

    a.append(int(change[i[0]]))
anneal_label = a

anneal_ = pd.DataFrame(anneal_data)
anneal_[38] = anneal_label
ann = anneal_

colum =  anneal_.columns


fea_con = [3,4,8,32,33,34,38]
fea_els = [i for i in colum if i not in fea_con]

anneal_ = anneal_[colum[fea_els]].astype(str)
anneal_ = pd.get_dummies(anneal_)
anneal_[colum[fea_con]] = ann[colum[fea_con]].values
cs = []
for j in range(20):
    cost_matrix = make_cost_matrix(anneal_label)
    anneal_train_set,anneal_test_set = train_test_split(anneal_,test_size=0.3)
    anneal_test_data = anneal_test_set.iloc[:,:-1].values
    anneal_test_label = anneal_test_set.iloc[:,-1].values
    
    t_data = anneal_train_set.iloc[:,:-1].values
    t_label = anneal_train_set.iloc[:,-1].values
    t_label = [int(i) for i in t_label]
    tree = DecisionTreeClassifier(criterion='entropy')
    tree.fit(t_data,t_label)
    
    predict_label = tree.predict(anneal_test_data)
    
    cs.append(compute_cs(cost_matrix,anneal_test_label,predict_label))
print("C4.5:",np.mean(cs))
print("C4.5:",np.std(cs))

# cs_=[]
# for j in range(20):
#     cost_matrix = make_cost_matrix(anneal_label)
#     anneal_train_set,anneal_test_set = train_test_split(anneal_,test_size=0.3)
#     anneal_test_data = anneal_test_set.iloc[:,:-1].values
#     anneal_test_label = anneal_test_set.iloc[:,-1].values
#     t_data,t_label = stratified_sampling(anneal_train_set,cost_matrix,'undersampling')
#     t_label = [int(i) for i in t_label]
#     tree = DecisionTreeClassifier(criterion='entropy')
#     tree.fit(t_data,t_label)
#     predict_label = tree.predict(anneal_test_data)
#     cs_.append(compute_cs(cost_matrix,anneal_test_label,predict_label))
# print("undersampling:",np.mean(cs_))
# print("undersamping",np.std(cs_))


# cs__=[]
# for j in range(20):
#     cost_matrix = make_cost_matrix(anneal_label)
#     anneal_train_set,anneal_test_set = train_test_split(anneal_,test_size=0.3)
#     anneal_test_data = anneal_test_set.iloc[:,:-1].values
#     anneal_test_label = anneal_test_set.iloc[:,-1].values
#     t_data,t_label = stratified_sampling(anneal_train_set,cost_matrix,'oversampling')
#     t_label = [int(i) for i in t_label]
#     tree = DecisionTreeClassifier(criterion='entropy')
#     tree.fit(t_data,t_label)
#     predict_label = tree.predict(anneal_test_data)
#     cs__.append(compute_cs(cost_matrix,anneal_test_label,predict_label))
# print("oversampling:",np.mean(cs__))
# print("voersamping",np.std(cs__))


# css_=[]
# for j in range(20):
#     cost_matrix = make_cost_matrix(anneal_label)
#     anneal_train_set,anneal_test_set = train_test_split(anneal_,test_size=0.3)
#     # print(anneal_train_set)
#     anneal_test_data = anneal_test_set.iloc[:,:-1].values
#     anneal_test_label = anneal_test_set.iloc[:,-1].values
#     tree = DecisionTreeClassifier(criterion='entropy')
#     meta = Metacost(anneal_train_set,tree,cost_matrix.values,q=False)
#     new_model = meta.fit(38)
#     predict_label  = new_model.predict(anneal_test_data)
#     css_.append(compute_cs(cost_matrix,anneal_test_label,predict_label))
# print("meta",np.mean(css_))
# print("meta",np.std(css_))

C4.5: 540.2409259259259
C4.5: 202.40074530893358


In [13]:
lenses = fetch_ucirepo(id=58) 
X = lenses.data.features.values
y = lenses.data.targets.values
unique = np.unique(y,return_counts=False)
change = {unique[i]:i for i in range(len(unique))}
# print(y)
y = [change[i[0]] for i in y]
lenses_ = pd.DataFrame(X)
lenses_['Y'] = y

cs = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    lenses_train_set,lenses_test_set = train_test_split(lenses_,test_size=0.3)
    lenses_test_data = lenses_test_set.iloc[:,:-1].values
    lenses_test_label = lenses_test_set.iloc[:,-1].values
    tree = D_Tree(lenses_train_set)
    tree.fit()
    predict_label = tree.predict(lenses_test_data)
    cs.append(compute_cs(cost_matrix,lenses_test_label,predict_label))
cs = np.array(cs)
print("C4.5",cs.mean())
print("C4.5",cs.std())

cs_ = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    lenses_train_set,lenses_test_set = train_test_split(lenses_,test_size=0.3)
    lenses_test_data = lenses_test_set.iloc[:,:-1].values
    lenses_test_label = lenses_test_set.iloc[:,-1].values
    train_set_x,train_set_y = stratified_sampling(lenses_train_set,cost_matrix.values,'undersampling') 
    train_set = pd.DataFrame(train_set_x)
    train_set['Y'] = train_set_y
    tree = D_Tree(train_set)
    tree.fit()
    predict_label = tree.predict(lenses_test_data)
    cs_.append(compute_cs(cost_matrix,lenses_test_label,predict_label))
cs_ = np.array(cs_)
# print(cs_)
print("under",np.mean(cs_))
print("under",np.std(cs_))

cs__ = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    lenses_train_set,lenses_test_set = train_test_split(lenses_,test_size=0.3)
    lenses_test_data = lenses_test_set.iloc[:,:-1].values
    lenses_test_label = lenses_test_set.iloc[:,-1].values
    train_set_x,train_set_y = stratified_sampling(lenses_train_set,cost_matrix.values,'oversampling') 
    train_set = pd.DataFrame(train_set_x)
    train_set['Y'] = train_set_y
    tree = D_Tree(train_set)
    tree.fit()
    predict_label = tree.predict(lenses_test_data)
    cs__.append(compute_cs(cost_matrix,lenses_test_label,predict_label))
cs__ = np.array(cs__)
# print(cs_)
print("over",np.mean(cs__))
print("over",np.std(cs__))

cs___=[]
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    lenses_train_set,wine_test_set = train_test_split(lenses_,test_size=0.3,random_state=42)
    lenses_test_data = lenses_test_set.iloc[:,:-1].values
    lenses_test_label = lenses_test_set.iloc[:,-1].values
    tree = DecisionTreeClassifier(criterion='entropy')
    meta = Metacost(lenses_train_set,tree,cost_matrix.values,q=False)
    model = meta.fit('Y')
    predict_label = model.predict(lenses_test_data)
    cs___.append(compute_cs(cost_matrix,lenses_test_label,predict_label))
cs___ = np.array(cs___)
# print(cs__)
print("meta",np.mean(cs___))
print("meta",np.std(cs___))

cs_k = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    lenses_train_set,lenses_test_set = train_test_split(lenses_,test_size=0.3,random_state=42)
    lenses_test_data = lenses_test_set.iloc[:,:-1].values
    lenses_test_label = lenses_test_set.iloc[:,-1].values
    knn = KNN(2,lenses_train_set,50)
    knn.fit(cost_matrix.values,n_neighbor=6,hamming_features=[i for i in range(lenses_train_set.shape[1]-1)])
    predict_label = knn.predict(wine_test_set)
    cs_k.append(compute_cs(cost_matrix,lenses_test_label,predict_label))
cs_k = np.array(cs_k)
print("my_algorithm",np.mean(cs_k))
print("my_algorithm",np.std(cs_k))

C4.5 989.96875
C4.5 502.9256342191035
under 702.4625
under 397.97504456152774
over 650.3625
over 353.98293104152634
meta 656.00625
meta 601.0089556776068
my_algorithm 408.73125
my_algorithm 117.89989314960171


In [14]:
# fetch dataset 
soybean_small = fetch_ucirepo(id=91) 
X = soybean_small.data.features.values 
y = soybean_small.data.targets.values
unique = np.unique(y,return_counts=False)
change = {unique[i]:i for i in range(len(unique))}
y = [change[i[0]] for i in y]
soybean_ = pd.DataFrame(X)
soybean_['Y'] = y


cs = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    soybean_train_set,soybean_test_set = train_test_split(soybean_,test_size=0.3)
    soybean_test_data = soybean_test_set.iloc[:,:-1].values
    soybean_test_label = soybean_test_set.iloc[:,-1].values
    tree = D_Tree(soybean_train_set)
    tree.fit()
    predict_label = tree.predict(soybean_test_data)
    cs.append(compute_cs(cost_matrix,soybean_test_label,predict_label))
cs = np.array(cs)
print("C4.5",cs.mean())
print("C4.5",cs.std())

cs_ = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    soybean_train_set,soybean_test_set = train_test_split(soybean_,test_size=0.3)
    soybean_test_data = soybean_test_set.iloc[:,:-1].values
    soybean_test_label = soybean_test_set.iloc[:,-1].values
    train_set_x,train_set_y = stratified_sampling(soybean_train_set,cost_matrix.values,'undersampling') 
    train_set = pd.DataFrame(train_set_x)
    train_set['Y'] = train_set_y
    tree = D_Tree(train_set)
    tree.fit()
    predict_label = tree.predict(soybean_test_data)
    cs_.append(compute_cs(cost_matrix,soybean_test_label,predict_label))
cs_ = np.array(cs_)
print("under",cs_.mean())
print("under",cs_.std())

cs__ = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    soybean_train_set,soybean_test_set = train_test_split(soybean_,test_size=0.3)
    soybean_test_data = soybean_test_set.iloc[:,:-1].values
    soybean_test_label = soybean_test_set.iloc[:,-1].values
    train_set_x,train_set_y = stratified_sampling(soybean_train_set,cost_matrix.values,'oversampling') 
    train_set = pd.DataFrame(train_set_x)
    train_set['Y'] = train_set_y
    tree = D_Tree(train_set)
    tree.fit()
    predict_label = tree.predict(soybean_test_data)
    cs__.append(compute_cs(cost_matrix,soybean_test_label,predict_label))
cs__ = np.array(cs__)
print("over",cs__.mean())
print("over",cs__.std())


cs___=[]
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    soybean_train_set,soybean_test_set = train_test_split(soybean_,test_size=0.3,random_state=42)
    soybean_test_data = soybean_test_set.iloc[:,:-1].values
    soybean_test_label = soybean_test_set.iloc[:,-1].values
    tree = DecisionTreeClassifier(criterion='entropy')
    meta = Metacost(soybean_train_set,tree,cost_matrix.values,q=False)
    model = meta.fit('Y')
    predict_label = model.predict(soybean_test_data)
    cs___.append(compute_cs(cost_matrix,soybean_test_label,predict_label))
cs___ = np.array(cs___)
# print(cs__)
print("meta",np.mean(cs___))
print("meta",np.std(cs___))

cs_k = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    soybean_train_set,soybean_test_set = train_test_split(soybean_,test_size=0.3,random_state=42)
    soybean_test_data = soybean_test_set.iloc[:,:-1].values
    soybean_test_label = soybean_test_set.iloc[:,-1].values
    knn = KNN(2,soybean_train_set,50)
    knn.fit(cost_matrix.values,n_neighbor=6,hamming_features=[i for i in range(soybean_train_set.shape[1]-1)])
    predict_label = knn.predict(soybean_test_set)
    cs_k.append(compute_cs(cost_matrix,soybean_test_label,predict_label))
cs_k = np.array(cs_k)
print("my_algorithm",np.mean(cs_k))
print("my_algorithm",np.std(cs_k))


C4.5 488.63
C4.5 129.07831640777883
under 447.74000000000007
under 168.40027962499877
over 530.7500000000001
over 163.56609792034806
meta 391.4733333333333
meta 105.7001353935851
my_algorithm 560.79
my_algorithm 215.110478359377


In [15]:
lung_cancer = fetch_ucirepo(id=62) 
X = lung_cancer.data.features.values
y = lung_cancer.data.targets.values
unique = np.unique(y,return_counts=False)
change = {unique[i]:i for i in range(len(unique))}
y = [change[i[0]] for i in y]
lung_ = pd.DataFrame(X)
lung_[56] = y
lun = lung_


fea_cate = [3,37]
fea_con = [i for i in range(lung_.shape[1]) if i not in fea_cate]

lung_ = lung_[fea_cate].astype(str)
lung_ = pd.get_dummies(lung_)
index = [i for i in range(lung_.shape[1],lung_.shape[1]+len(fea_con))]

lung_[index] = lun[fea_con].values

lung_ = lung_.rename(columns={62:'Y'})

cs = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    lung_train_set,lung_test_set = train_test_split(lung_,test_size=0.3)
    lung_test_data = lung_test_set.iloc[:,:-1].values
    lung_test_label = lung_test_set.iloc[:,-1].values
    tree = D_Tree(lung_train_set)
    tree.fit()
    predict_label = tree.predict(lung_test_data)
    predict_label = [int(i) for i in predict_label]
    cs.append(compute_cs(cost_matrix,lung_test_label,predict_label))
cs = np.array(cs)
print("C4.5",cs.mean())
print("C4.5",cs.std())

cs_ = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    lung_train_set,lung_test_set = train_test_split(lung_,test_size=0.3)
    lung_test_data = lung_test_set.iloc[:,:-1].values
    lung_test_label = lung_test_set.iloc[:,-1].values
    train_set_x,train_set_y = stratified_sampling(lung_train_set,cost_matrix.values,'undersampling') 
    train_set = pd.DataFrame(train_set_x)
    train_set['Y'] = train_set_y
    tree = D_Tree(train_set)
    tree.fit()
    predict_label = tree.predict(lung_test_data)
    predict_label = [int(i) for i in predict_label]
    cs_.append(compute_cs(cost_matrix,lung_test_label,predict_label))
cs_ = np.array(cs_)
print("undersampling",cs_.mean())
print("undersampling",cs_.std())

cs__ = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    lung_train_set,lung_test_set = train_test_split(lung_,test_size=0.3)
    lung_test_data = lung_test_set.iloc[:,:-1].values
    lung_test_label = lung_test_set.iloc[:,-1].values
    train_set_x,train_set_y = stratified_sampling(lung_train_set,cost_matrix.values,'oversampling') 
    train_set = pd.DataFrame(train_set_x)
    train_set['Y'] = train_set_y
    tree = D_Tree(train_set)
    tree.fit()
    predict_label = tree.predict(lung_test_data)
    predict_label = [int(i) for i in predict_label]
    cs__.append(compute_cs(cost_matrix,lung_test_label,predict_label))
cs__ = np.array(cs__)
print("oversampling",cs__.mean())
print("oversampling",cs__.std())


cs___=[]
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    lung_train_set,lung_test_set = train_test_split(lung_,test_size=0.3,random_state=42)
    lung_test_data = lung_test_set.iloc[:,:-1].values
    lung_test_label = lung_test_set.iloc[:,-1].values
    tree = DecisionTreeClassifier(criterion='entropy')
    meta = Metacost(lung_train_set,tree,cost_matrix.values,q=False)
    model = meta.fit('Y')
    predict_label = model.predict(lung_test_data)
    predict_label = [int(i) for i in predict_label]
    cs___.append(compute_cs(cost_matrix,lung_test_label,predict_label))
cs___ = np.array(cs___)
# print(cs__)
print("meta",np.mean(cs___))
print("meta",np.std(cs___))

cs_k = []
lun = lun.replace(np.nan,1000)
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    lung_train_set,lung_test_set = train_test_split(lun,test_size=0.3,random_state=42)
    lung_test_data = lung_test_set.iloc[:,:-1].values
    lung_test_label = lung_test_set.iloc[:,-1].values
    knn = KNN(2,lung_train_set,50)
    knn.fit(cost_matrix.values,n_neighbor=6,hamming_features=[i for i in range(lung_train_set.shape[1]-1)])
    predict_label = knn.predict(lung_test_set)
    cs_k.append(compute_cs(cost_matrix,lung_test_label,predict_label))
cs_k = np.array(cs_k)
print("my_algorithm",np.mean(cs_k))
print("my_algorithm",np.std(cs_k))

C4.5 807.835
C4.5 270.23318129904027
undersampling 745.3599999999999
undersampling 178.04232755162465
oversampling 831.6
oversampling 271.47579265930875
meta 632.88
meta 284.28659412642025
my_algorithm 605.66
my_algorithm 286.79842468186604


In [16]:
hepatitis = fetch_ucirepo(id=46) 
  
# data (as pandas dataframes) 
X = hepatitis.data.features.values 
y = hepatitis.data.targets.values
unique = np.unique(y,return_counts=False)
change = {unique[i]:i for i in range(len(unique))}
y = [change[i[0]] for i in y]
hep_ = pd.DataFrame(X)
hep_[19] = y
hep = hep_

fea_cate = [2,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
fea_con = [i for i in range(hep_.shape[1]) if i not in fea_cate]

hep_ = hep_[fea_cate].astype(str)
hep_ = pd.get_dummies(hep_)
index = [i for i in range(hep_.shape[1],hep_.shape[1]+len(fea_con))]

hep_[index] = hep[fea_con].values

hep_ = hep_.rename(columns={hep_.shape[1]-1:'Y'})


cs = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    hep_train_set,hep_test_set = train_test_split(hep_,test_size=0.3)
    hep_test_data = hep_test_set.iloc[:,:-1].values
    hep_test_label = hep_test_set.iloc[:,-1].values
    tree = D_Tree(hep_train_set)
    tree.fit()
    predict_label = tree.predict(hep_test_data)
    predict_label = [int(i) for i in predict_label]
    cs.append(compute_cs(cost_matrix,hep_test_label,predict_label))
cs = np.array(cs)
print("C4.5",cs.mean())
print("C4.5",cs.std())

cs_ = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    hep_train_set,hep_test_set = train_test_split(hep_,test_size=0.3)
    hep_test_data = hep_test_set.iloc[:,:-1].values
    hep_test_label = hep_test_set.iloc[:,-1].values
    train_set_x,train_set_y = stratified_sampling(hep_train_set,cost_matrix.values,'undersampling') 
    train_set = pd.DataFrame(train_set_x)
    train_set['Y'] = train_set_y
    tree = D_Tree(train_set)
    tree.fit()
    predict_label = tree.predict(hep_test_data)
    predict_label = [int(i) for i in predict_label]
    cs_.append(compute_cs(cost_matrix,hep_test_label,predict_label))
cs_ = np.array(cs_)
print("undersampling",cs_.mean())
print("undersampling",cs_.std())

# cs__ = []
# for i in range(20):
#     cost_matrix = make_cost_matrix(y)
#     hep_train_set,hep_test_set = train_test_split(hep_,test_size=0.3)
#     hep_test_data = hep_test_set.iloc[:,:-1].values
#     hep_test_label = hep_test_set.iloc[:,-1].values
#     train_set_x,train_set_y = stratified_sampling(hep_train_set,cost_matrix.values,'oversampling') 
#     train_set = pd.DataFrame(train_set_x)
#     train_set['Y'] = train_set_y
#     tree = D_Tree(train_set)
#     tree.fit()
#     predict_label = tree.predict(hep_test_data)
#     predict_label = [int(i) for i in predict_label]
#     cs__.append(compute_cs(cost_matrix,hep_test_label,predict_label))
# cs__ = np.array(cs__)
# print("oversampling",cs__.mean())
# print("oversampling",cs__.std())

cs___=[]
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    hep_train_set,lung_test_set = train_test_split(hep_,test_size=0.3,random_state=42)
    hep_test_data = hep_test_set.iloc[:,:-1].values
    hep_test_label = hep_test_set.iloc[:,-1].values
    tree = DecisionTreeClassifier(criterion='entropy')
    meta = Metacost(hep_train_set,tree,cost_matrix.values,q=False)
    model = meta.fit('Y')
    predict_label = model.predict(hep_test_data)
    predict_label = [int(i) for i in predict_label]
    cs___.append(compute_cs(cost_matrix,hep_test_label,predict_label))
cs___ = np.array(cs___)
# print(cs__)
print("meta",np.mean(cs___))
print("meta",np.std(cs___))

C4.5 848.2978723404254
C4.5 381.4059556270422
undersampling 526.9606382978724
undersampling 270.3064084913273
meta 435.2329787234042
meta 258.55962424195206


In [17]:
connectionist_bench_sonar_mines_vs_rocks = fetch_ucirepo(id=151) 
  
# data (as pandas dataframes) 
X = connectionist_bench_sonar_mines_vs_rocks.data.features.values 
y = connectionist_bench_sonar_mines_vs_rocks.data.targets.values

unique = np.unique(y,return_counts=False)
change = {unique[i]:i for i in range(len(unique))}
y = [change[i[0]] for i in y]
sonar_ = pd.DataFrame(X)
sonar_['Y'] = y


cs = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    sonar_train_set,sonar_test_set = train_test_split(sonar_,test_size=0.3)
    sonar_test_data = sonar_test_set.iloc[:,:-1].values
    sonar_test_label = sonar_test_set.iloc[:,-1].values
    tree = D_Tree(sonar_train_set)
    tree.fit()
    predict_label = tree.predict(sonar_test_data)
    cs.append(compute_cs(cost_matrix,sonar_test_label,predict_label))
cs = np.array(cs)
print("C4.5",cs.mean())
print("C4.5",cs.std())

cs_ = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    sonar_train_set,sonar_test_set = train_test_split(sonar_,test_size=0.3)
    sonar_test_data = sonar_test_set.iloc[:,:-1].values
    sonar_test_label = sonar_test_set.iloc[:,-1].values
    train_set_x,train_set_y = stratified_sampling(sonar_train_set,cost_matrix.values,'undersampling') 
    train_set = pd.DataFrame(train_set_x)
    train_set['Y'] = train_set_y
    tree = D_Tree(train_set)
    tree.fit()
    predict_label = tree.predict(sonar_test_data)
    cs_.append(compute_cs(cost_matrix,sonar_test_label,predict_label))
cs_ = np.array(cs_)
print("under",cs_.mean())
print("under",cs_.std())

cs__ = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    sonar_train_set,soybean_test_set = train_test_split(sonar_,test_size=0.3)
    sonar_test_data = sonar_test_set.iloc[:,:-1].values
    sonar_test_label = sonar_test_set.iloc[:,-1].values
    train_set_x,train_set_y = stratified_sampling(sonar_train_set,cost_matrix.values,'oversampling') 
    train_set = pd.DataFrame(train_set_x)
    train_set['Y'] = train_set_y
    tree = D_Tree(train_set)
    tree.fit()
    predict_label = tree.predict(sonar_test_data)
    cs__.append(compute_cs(cost_matrix,sonar_test_label,predict_label))
cs__ = np.array(cs__)
print("under",cs__.mean())
print("under",cs__.std())


cs___=[]
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    sonar_train_set,sonar_test_set = train_test_split(sonar_,test_size=0.3,random_state=42)
    sonar_test_data = sonar_test_set.iloc[:,:-1].values
    sonar_test_label = sonar_test_set.iloc[:,-1].values
    tree = DecisionTreeClassifier(criterion='entropy')
    meta = Metacost(sonar_train_set,tree,cost_matrix.values,q=False)
    model = meta.fit('Y')
    predict_label = model.predict(sonar_test_data)
    cs___.append(compute_cs(cost_matrix,sonar_test_label,predict_label))
cs___ = np.array(cs___)
# print(cs__)
print("meta",np.mean(cs___))
print("meta",np.std(cs___))

# lun = lun.replace(np.nan,1000)
cs_k = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    sonar_train_set,sonar_test_set = train_test_split(sonar_,test_size=0.3,random_state=42)
    sonar_test_data = sonar_test_set.iloc[:,:-1].values
    sonar_test_label = sonar_test_set.iloc[:,-1].values
    knn = KNN(6,sonar_train_set,50)
    knn.fit(cost_matrix.values,n_neighbor=20,euclidean_features=[i for i in range(sonar_train_set.shape[1]-1)])
    predict_label = knn.predict(sonar_test_set)
    cs_k.append(compute_cs(cost_matrix,sonar_test_label,predict_label))
cs_k = np.array(cs_k)
print("my_algorithm",np.mean(cs_k))
print("my_algorithm",np.std(cs_k))

C4.5 626.8174603174602
C4.5 221.98768328867257
under 654.6293650793651
under 158.7025545263123
under 521.8738095238095
under 196.79705584969017
meta 592.9198412698412
meta 220.38655539486936
my_algorithm 577.0761904761905
my_algorithm 268.7980790417749


In [36]:
pima = pd.read_csv('pima.dat',header=None)
x = pima.iloc[:,:-1].values
y = pima.iloc[:,-1].values
unique = np.unique(y)
change = {unique[i]:i for i in range(len(unique))}
y = [change[i] for i in y]
pima_ = pd.DataFrame(x)
pima_['Y'] = y

cs = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    pima_train_set,pima_test_set = train_test_split(pima_,test_size=0.3)
    pima_test_data = pima_test_set.iloc[:,:-1].values
    pima_test_label = pima_test_set.iloc[:,-1].values
    tree = D_Tree(pima_train_set)
    tree.fit()
    predict_label = tree.predict(pima_test_data)
    
    cs.append(compute_cs(cost_matrix,pima_test_label,predict_label))
cs = np.array(cs)
print("C4.5",cs.mean())
print("C4.5",cs.std())

cs_ = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    pima_train_set,sonar_test_set = train_test_split(pima_,test_size=0.3)
    pima_test_data = pima_test_set.iloc[:,:-1].values
    pima_test_label = pima_test_set.iloc[:,-1].values
    train_set_x,train_set_y = stratified_sampling(pima_train_set,cost_matrix.values,'undersampling') 
    train_set = pd.DataFrame(train_set_x)
    train_set['Y'] = train_set_y
    tree = D_Tree(train_set)
    tree.fit()
    predict_label = tree.predict(pima_test_data)
    cs_.append(compute_cs(cost_matrix,pima_test_label,predict_label))
cs_ = np.array(cs_)
print("under",cs_.mean())
print("under",cs_.std())

cs_ = []
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    pima_train_set,sonar_test_set = train_test_split(pima_,test_size=0.3)
    pima_test_data = pima_test_set.iloc[:,:-1].values
    pima_test_label = pima_test_set.iloc[:,-1].values
    train_set_x,train_set_y = stratified_sampling(pima_train_set,cost_matrix.values,'oversampling') 
    train_set = pd.DataFrame(train_set_x)
    train_set['Y'] = train_set_y
    tree = D_Tree(train_set)
    tree.fit()
    predict_label = tree.predict(pima_test_data)
    cs_.append(compute_cs(cost_matrix,pima_test_label,predict_label))
cs_ = np.array(cs_)
print("over",cs_.mean())
print("over",cs_.std())

cs___=[]
for i in range(20):
    cost_matrix = make_cost_matrix(y)
    pima_train_set,sonar_test_set = train_test_split(pima_,test_size=0.3,random_state=42)
    pima_test_data = pima_test_set.iloc[:,:-1].values
    pima_test_label = pima_test_set.iloc[:,-1].values
    tree = DecisionTreeClassifier(criterion='entropy')
    meta = Metacost(pima_train_set,tree,cost_matrix.values,q=False)
    model = meta.fit('Y')
    predict_label = model.predict(pima_test_data)
    cs___.append(compute_cs(cost_matrix,pima_test_label,predict_label))
cs___ = np.array(cs___)
# print(cs__)
print("meta",np.mean(cs___))
print("meta",np.std(cs___))

# cs_k = []
# for i in range(20):
#     cost_matrix = make_cost_matrix(y)
#     pima_train_set,pima_test_set = train_test_split(pima_,test_size=0.3,random_state=42)
#     pima_test_data = pima_test_set.iloc[:,:-1].values
#     pima_test_label = pima_test_set.iloc[:,-1].values
#     knn = KNN(6,pima_train_set,50)
    
#     knn.fit(cost_matrix.values,n_neighbor=20,euclidean_features=[i for i in range(pima_train_set.shape[1]-1)])
#     predict_label = knn.predict(pima_test_set)
#     cs_k.append(compute_cs(cost_matrix,pima_test_label,predict_label))
# cs_k = np.array(cs_k)
# print("my_algorithm",np.mean(cs_k))
# print("my_algorithm",np.std(cs_k))

C4.5 739.0515151515151
C4.5 169.54476287548417
under 594.2123376623377
under 149.96204061385873
over 433.3532467532467
over 177.9404864472333
meta 489.28744588744587
meta 168.4466260896014
